<a href="https://colab.research.google.com/github/jonghhhh/lecture_colabs/blob/main/naver_news_veideo_extract_033025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 네이버 뉴스 동영상 추출: selenium + 개발자도구 network(내적 API) 활용

1. Selenium을 사용해 네이버 뉴스 페이지를 열고 동영상 재생 버튼을 클릭합니다
2. 네트워크 트래픽을 모니터링하여 스트리밍 시작 시 전송되는 .ts 파일 URL을 찾습니다
3. 첫 번째 세그먼트(000000.ts)부터 순차적으로 모든 세그먼트를 다운로드합니다
4. 다운로드한 세그먼트를 하나의 파일로 결합하여 완전한 동영상 파일을 생성합니다
5. 작업이 완료되면 임시 파일을 자동으로 정리하고 다음 URL로 이동합니다

## Chrome 및 Chrome Driver 설치

In [ ]:
%%shell
# Chrome 및 Chrome Driver 설치
# 이 셀은 Colab에 필요한 모든 의존성을 설치합니다

# 시스템 패키지 업데이트
echo "===== 시스템 패키지 업데이트 중 ====="
sudo apt -y update
sudo apt install -y wget curl unzip

# libu2f-udev 설치 (Chrome이 필요로 함)
echo "===== libu2f-udev 설치 중 ====="
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb

# Chrome 브라우저 설치
echo "===== Chrome 브라우저 설치 중 ====="
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

# Chrome 버전 확인
echo "===== 설치된 Chrome 버전 ====="
google-chrome --version

# 최신 ChromeDriver 다운로드 및 설치
echo "===== ChromeDriver 설치 중 ====="
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
echo "ChromeDriver 버전: $CHROME_DRIVER_VERSION"
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver

# ChromeDriver 버전 확인
echo "===== 설치된 ChromeDriver 버전 ====="
chromedriver --version

# 필요한 Python 라이브러리 설치
echo "===== Python 라이브러리 설치 중 ====="
pip install selenium chromedriver-autoinstaller requests ipywidgets

echo "===== 설치 완료 ====="

## 저장 폴더 설정(Google Drive 연결 포함)

In [ ]:
# Google Drive 연결 (다운로드한 파일을 저장하기 위함)


# 다운로드 기본 경로 설정
import os

# 저장 경로 설정 (Google Drive 내 'naver_videos' 폴더)
DRIVE_PATH = 'naver_videos'

# 기본 저장 경로가 없으면 생성
if not os.path.exists(DRIVE_PATH):
    os.makedirs(DRIVE_PATH)
    print(f"✅ Google Drive에 저장 폴더 생성 완료: {DRIVE_PATH}")
else:
    print(f"✅ 이미 존재하는 저장 폴더를 사용합니다: {DRIVE_PATH}")

# 저장 경로 확인
print(f"📁 다운로드된 동영상은 다음 경로에 저장됩니다: {DRIVE_PATH}")

## 자장 파일명 설정: 뉴스 URL에서 고유 ID 추출

In [ ]:
def extract_news_id(news_url):
    """
    뉴스 URL에서 고유 ID를 추출합니다.

    설명:
    - 정규식을 사용해 URL에서 언론사 ID와 기사 ID를 추출합니다
    - 추출에 성공하면 '언론사ID_기사ID' 형식으로 반환합니다
    - 추출에 실패하면 타임스탬프 기반 임시 ID를 생성합니다

    Args:
        news_url (str): 네이버 뉴스 URL

    Returns:
        str: '언론사ID_기사ID' 형식의 고유 ID (예: '052_0002173010')
    """
    pattern = r'/article/(\d+)/(\d+)'
    match = re.search(pattern, news_url)
    if match:
        media_id = match.group(1)  # 언론사 ID (예: '052')
        article_id = match.group(2)  # 기사 ID (예: '0002173010')
        return f"{media_id}_{article_id}"
    else:
        # URL에서 추출 실패 시 타임스탬프를 사용한 임시 ID 생성
        return f"news_{int(time.time())}"

print("✅ 기본 설정 및 유틸리티 함수 로드 완료")

## 라이브러리 불러오기

In [ ]:
import sys
import time
import requests
import re
import os
import json
import tempfile
import shutil
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

## WebDriver 초기 설정: 함수화

In [ ]:
# Chrome WebDriver 경로 설정
print("🔧 Selenium WebDriver 환경 설정 중...")
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

def setup_driver():
    """
    셀레니움 웹드라이버를 설정하고 반환합니다.

    설명:
    - ChromeDriver 자동 설치를 실행합니다
    - 헤드리스 모드로 브라우저를 실행합니다 (화면에 보이지 않음)
    - 네트워크 트래픽 모니터링을 위한 성능 로깅을 활성화합니다
    - Colab 환경에 최적화된 설정을 적용합니다

    Returns:
        WebDriver: 설정된 Chrome 웹드라이버 인스턴스
    """
    # ChromeDriver 자동 설치
    print("ChromeDriver 자동 설치 중...")
    chromedriver_autoinstaller.install()

    # Chrome 옵션 설정
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # GUI 비활성화 (Colab에서는 필수)
    chrome_options.add_argument('--no-sandbox')  # 샌드박스 비활성화 (Colab에서 필요)
    chrome_options.add_argument('--disable-dev-shm-usage')  # 공유 메모리 사용 제한 비활성화
    chrome_options.add_argument('--window-size=1920,1080')  # 충분한 창 크기 설정

    # 성능 로깅 활성화 (네트워크 트래픽 모니터링용)
    chrome_options.set_capability("goog:loggingPrefs", {"performance": "ALL"})

    # WebDriver 초기화
    driver = webdriver.Chrome(options=chrome_options)
    print("✅ Chrome WebDriver 설정 완료")
    return driver

## ts 파일 URL 추출: TS URL 추출 함수

In [ ]:
def extract_ts_url(driver, news_url):
    """
    네이버 뉴스 페이지에서 .ts 파일 URL을 추출합니다.

    작동 방식:
    1. 뉴스 페이지를 로드합니다
    2. 다양한 선택자를 시도하여 동영상 재생 버튼을 찾고 클릭합니다
    3. 동영상이 재생되면 네트워크 트래픽을 분석합니다
    4. 첫 번째 세그먼트(000000.ts 또는 000001.ts)를 찾습니다
    5. URL을 반환하여 추후 다운로드에 사용합니다

    Args:
        driver (WebDriver): 셀레니움 웹드라이버 인스턴스
        news_url (str): 네이버 뉴스 URL

    Returns:
        str: 첫 번째 .ts 세그먼트 URL 또는 찾지 못한 경우 None
    """
    try:
        # 페이지 로드
        driver.get(news_url)
        time.sleep(3)  # 페이지 로드 대기

        # 진행 상황 표시
        print(f"📄 페이지 로드 완료: {news_url}")

        # 동영상 재생 버튼 찾기 (여러 선택자를 순차적으로 시도)
        selectors = [
            'button.pzp-button.pzp-brand-playback-button.pzp-pc__brand-playback-button',
            'span.pzp-ui-icon.pzp-brand-playback-button__icon',
            'svg.pzp-ui-icon__svg'
        ]

        # 재생 버튼 찾아 클릭
        button_clicked = False
        for selector in selectors:
            try:
                print(f"🔍 재생 버튼 선택자 시도: {selector}")
                play_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
                )
                play_button.click()
                print(f"✅ 재생 버튼 클릭 성공 (선택자: {selector})")
                button_clicked = True
                break
            except Exception as e:
                continue

        if not button_clicked:
            print("❌ 모든 재생 버튼 선택자가 실패했습니다.")
            return None

        print("▶️ 동영상 재생 시작됨")

        # 동영상 로딩 및 스트리밍 시작 대기
        # 충분한 시간을 주어야 .ts 파일 요청이 네트워크 로그에 기록됨
        print("⏳ 동영상 로딩 중... (5초 대기)")
        time.sleep(5)

        # 네트워크 로그에서 .ts 파일 URL 찾기
        print("🔍 네트워크 로그에서 .ts 파일 URL 검색 중...")
        logs = driver.get_log('performance')
        ts_url = None
        ts_url_count = 0

        for log in logs:
            try:
                log_data = json.loads(log['message'])
                message = log_data.get('message', {})

                if message.get('method') == 'Network.responseReceived':
                    request_url = message.get('params', {}).get('response', {}).get('url', '')

                    # .ts 파일 URL 필터링
                    if '.ts?' in request_url and 'naver-vod' in request_url:
                        ts_url_count += 1

                        # 000000.ts 또는 000001.ts 패턴 찾기 (첫 번째 세그먼트)
                        if '-000000.ts?' in request_url or '-000001.ts?' in request_url:
                            # 000001.ts를 000000.ts로 변경 (첫 세그먼트부터 확보)
                            ts_url = re.sub(r'-0*1\.ts\?', '-000000.ts?', request_url)
                            break

                        # 첫 번째 세그먼트를 아직 못 찾았다면 일단 발견된 .ts URL 저장
                        if not ts_url:
                            ts_url = request_url
            except:
                continue

        print(f"🔍 총 {ts_url_count}개의 .ts 파일 URL 발견")

        if ts_url:
            # 시작 세그먼트가 000000.ts인지 확인하고 조정
            if not re.search(r'-000000\.ts\?', ts_url):
                original_url = ts_url
                ts_url = re.sub(r'-(\d{6})\.ts\?', '-000000.ts?', ts_url)
                print(f"🔄 URL을 첫 번째 세그먼트(000000.ts)로 조정")
                print(f"  - 변경 전: ...{original_url[-30:].split('?')[0]}")
                print(f"  - 변경 후: ...{ts_url[-30:].split('?')[0]}")

            print(f"✅ .ts 파일 URL 추출 성공")
            return ts_url
        else:
            print("❌ 동영상 파일(.ts) URL을 찾지 못했습니다.")
            return None

    except Exception as e:
        print(f"❌ URL 추출 중 오류 발생: {str(e)}")
        return None

## ts파일 다운로드: TSDownloader 클래스

In [ ]:
class TSDownloader:
    """
    TS(Transport Stream) 세그먼트 다운로더 클래스

    이 클래스는 HLS(HTTP Live Streaming) 동영상의 .ts 세그먼트를 다운로드하고
    하나의 파일로 결합하는 기능을 제공합니다.

    주요 기능:
    - 멀티스레딩을 통한 병렬 다운로드
    - 세그먼트 자동 순번 관리
    - 다운로드 진행 상황 시각화
    - 임시 파일 자동 관리
    - 세그먼트 결합
    """

    def __init__(self, base_url, output_filename="combined_video.ts"):
        """
        TS 다운로더 초기화

        Args:
            base_url (str): 첫 번째 TS 세그먼트의 URL (.ts 파일 URL)
            output_filename (str): 최종 결합 파일명
        """
        self.base_url = base_url
        self.output_filename = output_filename
        self.segment_pattern = re.compile(r'(\d{6})\.ts')

        # 임시 디렉토리 생성 (세그먼트 파일을 저장할 위치)
        self.temp_dir = tempfile.mkdtemp()
        print(f"📁 임시 디렉토리 생성: {self.temp_dir}")

        # 진행 상황 표시용 위젯
        self.progress = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='다운로드:',
            bar_style='info',
            orientation='horizontal'
        )
        self.status_text = widgets.HTML(value="준비 중...")
        display(self.progress)
        display(self.status_text)

    def __del__(self):
        """소멸자: 임시 디렉토리 정리"""
        try:
            shutil.rmtree(self.temp_dir)
            print(f"🧹 임시 디렉토리 삭제됨: {self.temp_dir}")
        except:
            pass

    def generate_url(self, segment_number):
        """
        특정 세그먼트 번호에 대한 URL 생성

        Args:
            segment_number (int): 세그먼트 번호

        Returns:
            str: 해당 세그먼트의 URL
        """
        segment_str = f"{segment_number:06d}"  # 6자리 숫자로 포맷팅 (예: 000001)
        return self.segment_pattern.sub(f"{segment_str}.ts", self.base_url)

    def download_segment(self, segment_number, max_retries=3):
        """
        단일 세그먼트 다운로드

        Args:
            segment_number (int): 다운로드할 세그먼트 번호
            max_retries (int): 최대 재시도 횟수

        Returns:
            bool: 다운로드 성공 여부
        """
        url = self.generate_url(segment_number)
        output_path = os.path.join(self.temp_dir, f"{segment_number:06d}.ts")

        for attempt in range(max_retries):
            try:
                # HTTP 요청으로 세그먼트 다운로드
                response = requests.get(url, timeout=10)

                # 응답 코드 확인
                if response.status_code == 200:
                    # 콘텐츠가 유효한지 확인 (너무 작은 파일은 오류 응답일 수 있음)
                    if len(response.content) > 1000:  # 1KB 이상
                        # 파일로 저장
                        with open(output_path, 'wb') as f:
                            f.write(response.content)
                        # 진행 상황 업데이트
                        self.status_text.value = f"세그먼트 {segment_number:06d} 다운로드 완료"
                        return True
                    else:
                        print(f"⚠️ 세그먼트 {segment_number:06d}가 너무 작습니다. 유효하지 않을 수 있음")
                        return False
                else:
                    # 404 오류는 일반적으로 더 이상 세그먼트가 없음을 의미
                    if response.status_code == 404:
                        print(f"🛑 세그먼트 {segment_number:06d} 다운로드 실패: 상태 코드 404 (세그먼트 끝)")
                    else:
                        print(f"❌ 세그먼트 {segment_number:06d} 다운로드 실패: 상태 코드 {response.status_code}")

                    if attempt < max_retries - 1:
                        time.sleep(1)  # 재시도 전 대기
            except Exception as e:
                print(f"❌ 세그먼트 {segment_number:06d} 다운로드 중 오류: {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(1)  # 재시도 전 대기

        return False

    def download_segments(self, start_segment=0, max_failures=5, max_workers=5):
        """
        연속적으로 TS 세그먼트 다운로드

        멀티스레딩을 사용하여 여러 세그먼트를 동시에 다운로드합니다.
        연속된 실패가 설정 횟수(max_failures)를 초과하면 다운로드를 중단합니다.
        이는 동영상의 끝에 도달했음을 의미할 수 있습니다.

        Args:
            start_segment (int): 시작 세그먼트 번호 (일반적으로 0)
            max_failures (int): 중단 전 허용되는 연속 실패 횟수
            max_workers (int): 동시 다운로드 스레드 수

        Returns:
            int: 성공적으로 다운로드된 세그먼트 수
        """
        segment_number = start_segment
        consecutive_failures = 0
        successful_downloads = 0

        print(f"▶️ 세그먼트 {start_segment}부터 다운로드 시작")
        self.status_text.value = f"세그먼트 {start_segment}부터 다운로드 시작"

        # ThreadPoolExecutor를 사용한 병렬 다운로드
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            while consecutive_failures < max_failures:
                # 세그먼트 배치를 동시에 다운로드
                batch_size = min(max_workers, max_failures)
                future_to_segment = {
                    executor.submit(self.download_segment, segment_number + i): segment_number + i
                    for i in range(batch_size)
                }

                # 결과 처리
                batch_success = False
                for future in future_to_segment:
                    segment_num = future_to_segment[future]
                    if future.result():  # 다운로드 성공
                        batch_success = True
                        successful_downloads += 1
                        consecutive_failures = 0
                        # 진행 상황 업데이트 (대략적인 추정, 동영상 길이를 모르므로)
                        self.progress.value = min(100, int(successful_downloads / 100 * 100))
                    else:  # 다운로드 실패
                        consecutive_failures += 1
                        if consecutive_failures >= max_failures:
                            print(f"🛑 연속 {max_failures}회 실패로 다운로드 중단 (동영상 끝 도달)")
                            break

                # 배치 전체가 실패하면 중지
                if not batch_success:
                    break

                # 다음 배치로 이동
                segment_number += batch_size

        # 진행 상황 100% 표시
        self.progress.value = 100
        self.status_text.value = f"✅ 다운로드 완료. 총 {successful_downloads}개 세그먼트 다운로드 성공."
        print(f"✅ 다운로드 완료. 총 {successful_downloads}개 세그먼트 다운로드 성공.")
        return successful_downloads

    def combine_segments(self, num_segments, output_path):
        """
        다운로드된 TS 세그먼트를 하나의 파일로 결합

        Args:
            num_segments (int): 결합할 세그먼트 수
            output_path (str): 결합된 파일의 저장 경로

        Returns:
            bool: 결합 성공 여부
        """
        try:
            # 출력 디렉토리 확인 및 생성
            output_dir = os.path.dirname(output_path)
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir)

            self.status_text.value = "🔄 파일 결합 중..."
            print("🔄 세그먼트 파일 결합 중...")

            # 모든 세그먼트를 순서대로 하나의 파일로 결합
            with open(output_path, 'wb') as outfile:
                for i in range(0, num_segments):
                    segment_path = os.path.join(self.temp_dir, f"{i:06d}.ts")
                    if os.path.exists(segment_path):
                        with open(segment_path, 'rb') as infile:
                            outfile.write(infile.read())
                    else:
                        print(f"⚠️ 경고: 세그먼트 {i:06d}.ts 파일이 없습니다.")

            # 파일 크기 확인
            file_size_mb = os.path.getsize(output_path) / (1024 * 1024)

            self.status_text.value = f"✅ {num_segments}개의 세그먼트를 {output_path}로 결합 완료 (크기: {file_size_mb:.2f} MB)"
            print(f"✅ {num_segments}개의 세그먼트를 {output_path}로 결합 완료")
            print(f"📊 파일 크기: {file_size_mb:.2f} MB")
            return True
        except Exception as e:
            self.status_text.value = f"❌ 파일 결합 중 오류 발생: {e}"
            print(f"❌ 파일 결합 중 오류 발생: {e}")
            return False

    def run(self, output_path):
        """
        전체 다운로드 및 결합 프로세스 실행

        Args:
            output_path (str): 최종 결합 파일 저장 경로

        Returns:
            str: 성공 시 출력 파일 경로, 실패 시 None
        """
        try:
            # 세그먼트 다운로드 (0번부터 시작)
            num_segments = self.download_segments()

            if num_segments > 0:
                # 세그먼트 결합
                if self.combine_segments(num_segments, output_path):
                    return output_path

            self.status_text.value = "❌ 동영상 다운로드에 실패했습니다."
            print("❌ 동영상 다운로드에 실패했습니다.")
            return None
        except Exception as e:
            self.status_text.value = f"❌ 다운로드 프로세스 중 오류 발생: {e}"
            print(f"❌ 다운로드 프로세스 중 오류 발생: {e}")
            return None

## 다중 비디오 다운로드

In [ ]:
def download_multiple_videos(urls, output_folder=DRIVE_PATH):
    """
    여러 뉴스 URL에서 동영상을 연속적으로 다운로드합니다.

    작동 방식:
    1. 각 URL에 대해 하나씩 처리합니다
    2. 뉴스 ID를 추출하여 파일명으로 사용합니다 (예: '052_0002173010.ts')
    3. 이미 다운로드된 파일은 건너뜁니다 (중복 방지)
    4. 모든 URL 처리 후 드라이버를 한 번만 종료합니다

    Args:
        urls (list): 네이버 뉴스 URL 리스트
        output_folder (str): 다운로드된 파일을 저장할 폴더

    Returns:
        list: 다운로드된 파일 경로 리스트
    """
    # 출력 폴더 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"📁 출력 폴더 생성: {output_folder}")

    # 전체 진행 상황 표시 위젯
    total_progress = widgets.IntProgress(
        value=0,
        min=0,
        max=len(urls),
        description='전체 진행:',
        bar_style='success',
        orientation='horizontal'
    )
    status_html = widgets.HTML(value=f"🔍 총 {len(urls)}개 URL 처리 예정")
    display(total_progress)
    display(status_html)

    print(f"🌐 {len(urls)}개 뉴스 URL 처리 시작")

    # 드라이버 초기화 (모든 URL 처리 후 한 번만 종료)
    driver = setup_driver()
    downloaded_files = []

    try:
        for i, news_url in enumerate(urls):
            # 진행 상황 업데이트
            clear_output(wait=True)  # 출력 정리
            display(total_progress)
            display(status_html)

            # 현재 작업 중인 URL 정보 표시
            total_progress.value = i
            status_html.value = f"[{i+1}/{len(urls)}] 처리 중: {news_url}"
            print(f"\n📰 [{i+1}/{len(urls)}] 뉴스 URL 처리 중: {news_url}")

            # 뉴스 ID 추출하여 파일명 생성 (예: '052_0002173010.ts')
            news_id = extract_news_id(news_url)
            output_path = os.path.join(output_folder, f"{news_id}.ts")

            # 이미 다운로드된 파일이 있는지 확인 (중복 방지)
            if os.path.exists(output_path):
                print(f"⏩ 파일이 이미 존재합니다: {output_path}")
                downloaded_files.append(output_path)
                continue

            # TS URL 추출
            ts_url = extract_ts_url(driver, news_url)

            if not ts_url:
                print(f"❌ URL {news_url}에서 동영상을 찾을 수 없습니다.")
                continue

            # 다운로드 실행
            print(f"⬇️ 동영상 다운로드 시작: {output_path}")
            downloader = TSDownloader(ts_url)
            result_file = downloader.run(output_path)

            if result_file:
                print(f"✅ 다운로드 완료: {result_file}")
                downloaded_files.append(result_file)
            else:
                print(f"❌ URL {news_url}에서 동영상 다운로드 실패")

        # 진행 상황 완료 업데이트
        total_progress.value = len(urls)
        status_html.value = f"✅ 모든 URL 처리 완료! {len(downloaded_files)}/{len(urls)} 파일 다운로드 성공"

    except Exception as e:
        status_html.value = f"❌ 다운로드 프로세스 중 오류 발생: {e}"
        print(f"❌ 다운로드 프로세스 중 오류 발생: {e}")

    finally:
        # 모든 URL 처리 후 드라이버 종료
        print("🔄 모든 뉴스 URL 처리 완료, 드라이버 종료")
        driver.quit()

    return downloaded_files

## 실행

In [ ]:
# 다운로드할 네이버 뉴스 URL 리스트

news_urls = [
        "https://n.news.naver.com/mnews/article/052/0002173010?sid=102",
        "https://n.news.naver.com/mnews/article/056/0011921345?sid=101",
        "https://n.news.naver.com/mnews/article/055/0001244569?sid=102",
        "https://n.news.naver.com/mnews/article/448/0000517031?sid=102",
        "https://n.news.naver.com/mnews/article/449/0000304011?sid=102",
        "https://n.news.naver.com/mnews/article/437/0000435189?sid=102"
]

# 다운로드 폴더 설정
output_folder = 'naver_news_videos1'

import os
# 기본 저장 경로가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"✅ Google Drive에 저장 폴더 생성 완료: {output_folder}")
else:
    print(f"✅ 이미 존재하는 저장 폴더를 사용합니다: {output_folder}")

print(f"🎬 {len(news_urls)}개 네이버 뉴스 동영상 다운로드 시작")

# 연속 다운로드 실행
downloaded_files = download_multiple_videos(news_urls[3:4], output_folder)

# 결과 출력
print("\n====== 다운로드 결과 요약 ======")
if downloaded_files:
    print(f"✅ 총 {len(downloaded_files)}/{len(news_urls)} 파일 다운로드 완료:")

    # 다운로드된 파일 목록 표시
    for file_path in downloaded_files:
        file_name = os.path.basename(file_path)
        if os.path.exists(file_path):
            file_size = f"{os.path.getsize(file_path) / (1024 * 1024):.2f} MB"
            print(f" - {file_name} ({file_size})")
        else:
            print(f" - {file_name} (파일 없음)")

    # 파일 목록 HTML 테이블로 표시
    html_content = """
    <h3>다운로드된 파일 목록</h3>
    <table style="width:100%; border-collapse:collapse;">
      <tr style="background-color:#f0f0f0;">
        <th style="padding:8px; text-align:left; border:1px solid #ddd;">파일명</th>
        <th style="padding:8px; text-align:left; border:1px solid #ddd;">경로</th>
        <th style="padding:8px; text-align:left; border:1px solid #ddd;">크기</th>
      </tr>
    """

    for file_path in downloaded_files:
        file_name = os.path.basename(file_path)
        file_dir = os.path.dirname(file_path)
        if os.path.exists(file_path):
            file_size = f"{os.path.getsize(file_path) / (1024 * 1024):.2f} MB"
        else:
            file_size = "파일 없음"

        html_content += f"""
        <tr>
          <td style="padding:8px; border:1px solid #ddd;">{file_name}</td>
          <td style="padding:8px; border:1px solid #ddd;">{file_dir}</td>
          <td style="padding:8px; border:1px solid #ddd;">{file_size}</td>
        </tr>
        """

    html_content += "</table>"
    display(HTML(html_content))

    print("\n📋 다운로드 통계:")
    print(f" - 성공률: {len(downloaded_files)/len(news_urls)*100:.1f}%")
    print(f" - 총 동영상 수: {len(downloaded_files)}개")

    total_size = sum([os.path.getsize(f) for f in downloaded_files if os.path.exists(f)])
    print(f" - 총 용량: {total_size/(1024*1024):.2f} MB")

    print(f"\n📂 다운로드된 파일은 Google Drive 폴더에서 확인할 수 있습니다:")
    print(f" - 경로: {output_folder}")
else:
    print("❌ 다운로드된 파일이 없습니다.")

print("\n💡 참고: 404 오류는 동영상 세그먼트의 끝을 나타내는 정상적인 신호입니다.")